In [1]:
from utils import *
import pandas as pd

#### Extract Bills & Summaries

In [2]:
# Bills
INDIR_115_HR = './data/115/bills/hr'
INDIR_115_S = './data/115/bills/s'

# Concurrent Resolutions
INDIR_115_HCONRES = './data/115/bills/hconres'
INDIR_115_SCONRES = './data/115/bills/sconres'

# Joint Resolutions
INDIR_115_HJRES = './data/115/bills/hjres'
INDIR_115_SJRES = './data/115/bills/sjres'

# Simple Resolutions
INDIR_115_HRES = './data/115/bills/hres'
INDIR_115_SRES = './data/115/bills/sres'

INDIR_ALL = [INDIR_115_HR, INDIR_115_S, 
              INDIR_115_HCONRES, INDIR_115_SCONRES, 
              INDIR_115_HJRES, INDIR_115_SJRES, 
              INDIR_115_HRES, INDIR_115_SRES
             ]

In [3]:
data = []

for i in INDIR_ALL:
    print('Processing {}'.format(i))
    for d in walk_dirs(i):
        data.append(d)
        
df_115 = pd.DataFrame(data)

Processing ./data/115/bills/hr
Processing ./data/115/bills/s
Processing ./data/115/bills/hconres
Processing ./data/115/bills/sconres
Processing ./data/115/bills/hjres
Processing ./data/115/bills/sjres
Processing ./data/115/bills/hres
Processing ./data/115/bills/sres


In [4]:
df_115.head()

,Directory,Number,Subtype,Summary,Type,Version
0,./data/115/bills/hr/hr1/text-versions/ih,HR1,HR,1,Bills,IH
1,./data/115/bills/hr/hr10/text-versions/eh,HR10,HR,1,Bills,EH
2,./data/115/bills/hr/hr10/text-versions/ih,HR10,HR,1,Bills,IH
3,./data/115/bills/hr/hr10/text-versions/rfs,HR10,HR,1,Bills,RFS
4,./data/115/bills/hr/hr10/text-versions/rh,HR10,HR,1,Bills,RH


#### Summary Stats

In [5]:
print('Number of rows: {}'.format(len(df_115)))
print('Number of unique bills: {}'.format(len(df_115.Number.unique())))

Number of rows: 9390
Number of unique bills: 7626


When Version = N/A it means there are no bill texts, so these are not usable.

In [6]:
with_bill_text = df_115[df_115.Version != 'N/A']
print('Number of unique bills with bill text: {}'.format(len(with_bill_text.Number.unique())))

Number of unique bills with bill text: 7548


We are only interested in bills that have at least one summary.

In [7]:
with_summary = with_bill_text[with_bill_text.Summary > 0]
print('Number of unique bills with bill text and at least one summary: {}'.format(len(with_summary.Number.unique())))

Number of unique bills with bill text and at least one summary: 5192


How many of each type are there?

In [8]:
with_summary.groupby(['Type', 'Subtype']).size()

Type                    Subtype
Bills                   HR         3872
                        S          1560
Concurrent Resolutions  HCONRES     118
                        SCONRES      45
Joint Resolutions       HJRES       173
                        SJRES        68
Simple Resolutions      HRES        631
                        SRES        332
dtype: int64

Since there are multiple versions of each bill, choose the most recent one.

In [9]:
with_summary['to_use'] = 0
recents_marked = get_recent_bills(with_summary)
unique_bills = recents_marked[recents_marked.to_use == 1]

/Users/stephaniedoctor/anaconda/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/stephaniedoctor/Documents/Columbia/Capstone/bill-summarization/utils.py:314: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = with_summary[with_summary.Number == bill_no][with_summary.Version != 'EAS']
/Users/stephaniedoctor/anaconda/envs/py3k/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [11]:
unique_bills.head()

,Directory,Number,Subtype,Summary,Type,Version,to_use
0,./data/115/bills/hr/hr1/text-versions/ih,HR1,HR,1,Bills,IH,1
1,./data/115/bills/hr/hr10/text-versions/eh,HR10,HR,1,Bills,EH,1
5,./data/115/bills/hr/hr100/text-versions/ih,HR100,HR,1,Bills,IH,1
6,./data/115/bills/hr/hr1000/text-versions/ih,HR1000,HR,1,Bills,IH,1
7,./data/115/bills/hr/hr1001/text-versions/ih,HR1001,HR,1,Bills,IH,1


In [12]:
unique_bills.Version.value_counts()

IH     3063
IS     1364
EH      444
ATS     146
ENR      96
ES       54
RH       12
PCS      12
CPS       1
Name: Version, dtype: int64